In [3]:
#This is a notebook to test the DistilBERT model
import tensorflow as tf
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import pandas as pd
import numpy as np

In [18]:
df=pd.read_csv('Reviews.csv')
df.head()

In [20]:
df_copy=df.copy()
df_copy.drop(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Time', 'Summary'],
             inplace=True, axis=1)
df_copy.head()

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...


In [21]:
df_copy["Sentiment"] = df_copy["Score"].apply(lambda score: "positive" if score >= 3 else ("negative" if score <= 2 else "neutral"))
df_copy['Sentiment'] = df_copy['Sentiment'].map({'positive':1, 'negative':0, 'neutral':2})

In [22]:
df_copy.head()

,Score,Text,Sentiment
0,5,I have bought several of the Vitality canned d...,1
1,1,Product arrived labeled as Jumbo Salted Peanut...,0
2,4,This is a confection that has been around a fe...,1
3,2,If you are looking for the secret ingredient i...,0
4,5,Great taffy at a great price. There was a wid...,1


In [23]:
data=df_copy[["Sentiment","Text"]]
data.head()
data.shape

(568454, 2)

In [24]:
data = data.head(1000)

In [25]:
reviews = data['Text'].values.tolist()
labels = data['Sentiment'].tolist()

In [26]:
from sklearn.model_selection import train_test_split
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=.2)

In [27]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
tokenizer([training_sentences[0]], truncation=True, padding=True, max_length=128)

{'input_ids': [[101, 1996, 5510, 1997, 2122, 2317, 18178, 25062, 2099, 4257, 7852, 2015, 2003, 2066, 1037, 3180, 8579, 2121, 1011, 2029, 2003, 2025, 2919, 1010, 3272, 2008, 1045, 4149, 2068, 2138, 1045, 2359, 1037, 8808, 5510, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 2054, 2001, 1037, 4121, 10520, 1029, 2129, 22369, 1996, 14793, 1997, 1996, 3482, 2003, 1012, 1996, 6302, 2006, 1996, 3482, 1006, 1045, 4149, 2122, 1999, 3573, 1007, 3084, 2009, 2298, 2066, 2009, 2003, 2440, 1997, 2146, 4257, 27035, 2015, 1006, 9186, 1996, 3091, 1998, 9381, 1997, 1996, 3482, 1007, 1012, 3308, 999, 1996, 6081, 11851, 2008, 4324, 1996, 8579, 2545, 2003, 2055, 1016, 1000, 3760, 2035, 2105, 1011, 2975, 2017, 2007, 2055, 2321, 2030, 2061, 2235, 4257, 27035, 2015, 1012, 1026, 7987, 1013, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [28]:
train_encodings = tokenizer(training_sentences,truncation=True,padding=True)
val_encodings = tokenizer(validation_sentences,truncation=True,padding=True)

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),training_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings),validation_labels))

In [29]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [30]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

In [31]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])


In [32]:
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=3,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/3
50/50 [==============================] - 723s 14s/step - loss: 0.3866 - accuracy: 0.8487 - val_loss: 0.3372 - val_accuracy: 0.8600
Epoch 2/3
50/50 [==============================] - 731s 15s/step - loss: 0.2296 - accuracy: 0.9175 - val_loss: 0.3175 - val_accuracy: 0.8650
Epoch 3/3
50/50 [==============================] - 710s 14s/step - loss: 0.0944 - accuracy: 0.9762 - val_loss: 0.3109 - val_accuracy: 0.9100


In [40]:
model.save_pretrained("./sentiment")

In [85]:
from transformers import TFDistilBertForSequenceClassification

# Load the Hugging Face Transformers model
transformers_model = TFDistilBertForSequenceClassification.from_pretrained(
    r"C:\Users\Asus\Desktop\Submission\sentiment\tf_model.h5",
    config=r"C:\Users\Asus\Desktop\Submission\sentiment\config.json"
)

Some layers from the model checkpoint at C:\Users\Asus\Desktop\Submission\sentiment\tf_model.h5 were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at C:\Users\Asus\Desktop\Submission\sentiment\tf_model.h5 and are newly initialized: ['dropout_199']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferenc

In [86]:
#Check model prediction

def senti(test_sentence):
    try:
        predict_input = tokenizer.encode(test_sentence,
                                          truncation=True,
                                          padding=True,
                                          return_tensors="tf")
        tf_output = transformers_model.predict(predict_input)[0]
        tf_prediction = tf.nn.softmax(tf_output, axis=1)
        labels = ['Negative', 'Positive', 'Neutral']
        label_index = tf.argmax(tf_prediction, axis=1).numpy()[0]
        predicted_label = labels[label_index]
        return predicted_label
    except Exception as e:
        print("Error:", str(e))
        return None

In [88]:
test_sentence = "I love this product"
test_sentence1 = "This is worst food"
senti(test_sentence)
senti(test_sentence1)

1/1 [==============================] - 0s 35ms/step


'Negative'

I have fine-tuned the model using the following hyperparameters:
- batch_size = 16
- learning_rate = 5e-5
- epochs = 3
- epsilon=1e-08
- optimizer = Adam
- metrics = accuracy
- validation_split = 0.2
- shuffle = True


I have fine-tuned the model for 1000 data points only and the model is performing well on the test data. But the model can be further fine-tuned for better performance.